In [1]:
# Import libraries
import os
import warnings

import numpy as np
import pandas as pd

import gc  # Garbage collector


warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

from xgboost import XGBClassifier

In [3]:
# load the model
import joblib
xgb_classifier = joblib.load("../input/01-starter-xgboost-implementation/xgb_classifier_v1.h5")

In [4]:
oe = joblib.load("../input/01-starter-xgboost-implementation/oe.h5")

In [5]:
test = pd.read_feather('../input/amexfeather/test_data.ftr')#.iloc[6000000:]

In [6]:
# Handling date column

test["S_2_day"] = test["S_2"].dt.day
test["S_2_month"] = test["S_2"].dt.month
test["S_2_year"] = test["S_2"].dt.year

In [7]:
drop_cols = ['D_87', 'D_88', 'D_108', 'D_111', 'D_110', 'B_39', 'D_73', 'B_42', 'D_136',
 'D_138', 'D_137', 'D_135', 'D_134', 'R_9', 'B_29', 'D_106', 'D_132', 'D_49',
 'R_26', 'D_76', 'D_66', 'D_42', 'D_142', 'D_53', 'D_82','S_2']

test.drop(columns = drop_cols,axis=1, inplace=True)

# converting pandas "categorical" dtype to numeric
cols = ["D_68", "B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126"]
test[cols] = test[cols].apply(pd.to_numeric, errors='coerce')

In [8]:
categorical_columns = ["D_63","D_64"]
test_enc = oe.transform(test[categorical_columns])
test[categorical_columns] = test_enc

In [9]:
_ = gc.collect()

In [10]:
# considering only one data point per customer (latest one) as time series is not being used
test = test.groupby(['customer_ID']).nth(-1).reset_index(drop=True)

In [11]:
# test.isna().sum()

In [12]:
# # For numeric columns
# numeric_columns = test.select_dtypes(np.number).columns
# test[numeric_columns] = test[numeric_columns].fillna(test[numeric_columns].mean())

In [13]:
# Score up the test dataset
test_preds = xgb_classifier.predict(test)
# test_preds.view()


### Submission

In [14]:
# Make submission
sub_data = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')
sub_data.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0


In [15]:
sub_data.shape

(924621, 2)

In [16]:
sub_data['prediction'] = test_preds
sub_data.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1


In [17]:
# Submission file
sub_data.to_csv('submission.csv', index=False)